## Concatenating all attention distributions 

We extract the attention distribution directly from a model that has been pretrained for bias detection! We then using windowing to extract a window of attention scores around the biased word. In this experiment we just use a very simple logistic regression model from sklearn - this is so that we can look at the weights more directly.

In [1]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import AttentionExperiment, ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params
from src.utils.attention_utils import reduce_attention_dist, return_idx_attention_dist, window_attention_dist
from src.utils.classification_utils import run_bootstrapping
from src.utils.shared_utils import get_bias_predictions

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
params = Params.read_params("linear-params-window.json")
print("model = {}".format(params.final_task['model']))
print("layers = {}".format(params.intermediary_task["attention"]["layers"]))
print("reducer = {}".format(params.intermediary_task["attention"]["reducer"]))

model = shallow_nn
layers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
reducer = concat


In [4]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dev_dataset = ExperimentDataset.init_dataset(params.dataset)

04/10/2020 08:42:14 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
100it [00:00, 4070.32it/s]


In [5]:
import pickle 
train_dataset = pickle.load(open(params.dataset["weakly_labeled_data"], "rb"))

### Attention Experiment: 
* Is a class that wraps useful methods to extract attention distributions from a given BERT-based model 
* The user has to provide in two config files: One to specify parameters for how the attention scores should be extracted and combined, and other to specify the intermediary model from which the attention scores should be extracted from
* The user needs to instantiate the attention experiment with a function that tells the model how to run 
 inference on the given model. The function header is specified below: 
 
 ``` def initialize_attention_experiment(cls, intermediary_task_params, dataset_params, verbose=False) ```
 


In [6]:
attention_experiment = AttentionExperiment.initialize_attention_experiment(params.intermediary_task, params.dataset, verbose=True, from_pretrained=True)

04/10/2020 08:42:20 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/10/2020 08:42:20 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/10/2020 08:42:20 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmp4727q9gs
04/10/2020 08:42:25 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

Instantiated joint model with pretrained weights.
Succesfully loaded in attention experiment!


In [7]:
attention_dataloader_dev = dev_dataset.return_dataloader(batch_size=params.intermediary_task['attention']['attention_extraction_batch_size'])  
attention_dataloader_train = train_dataset.return_dataloader(batch_size=params.intermediary_task['attention']['attention_extraction_batch_size'])

```extract_attention_scores()``` works out of the box because the attention experiment has the config file saved, and knows what BERT model to use/load in, which layers to extract the attention scores from, and what the inference function is that should be used on this particular BERT model.

Attention_scores is then a list of dictionaries. The keys in this dictionary are the specific layers of a BERT model and the values are the corresponding attention distributions extracted from that particular layer.

In [8]:
attention_scores_dev = attention_experiment.extract_attention_scores(attention_dataloader_dev)

In [9]:
# Saving out attention weights from the train dataset to facilitate future runs
import os 
attention_weights_file = "model_weights/attention_scores_train.pkl"
if os.path.exists(attention_weights_file):
    print("Loading in existing train attention weights")
    attention_scores_train = pickle.load(open(attention_weights_file, "rb"))
else:
    print("Generating new training attention weights ")
    if (not os.path.isdir("model_weights")):
            os.mkdir("model_weights")
    attention_scores_train = attention_experiment.extract_attention_scores(attention_dataloader_train)
    pickle.dump(attention_scores_train, open(attention_weights_file, "wb+"))

Loading in existing train attention weights


Getting the predictions from the BERT model trained to detect bias, and using those to index into the attention scores

In [10]:
bias_predictions_train = get_bias_predictions(train_dataset, params.intermediary_task, params.dataset, batch_size=32)
bias_predictions_dev = get_bias_predictions(dev_dataset, params.intermediary_task, params.dataset, batch_size=32)

04/10/2020 08:46:32 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/10/2020 08:46:32 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/10/2020 08:46:32 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmp0_ff954t
04/10/2020 08:46:36 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

04/10/2020 08:51:24 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
04/10/2020 08:51:25 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
04/10/2020 08:51:25 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpmc9h835t
04/10/2020 08:51:28 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_d

In [11]:
bias_indices_train = torch.argmax(bias_predictions_train == 1, dim=1).tolist()
bias_indices_dev = torch.argmax(bias_predictions_dev == 1, dim=1).tolist()

In [12]:
attention_scores_indexed_train = return_idx_attention_dist(attention_scores_train, bias_indices_train)
reduced_attention_train = reduce_attention_dist(attention_scores_indexed_train, params.intermediary_task["attention"])
windowed_dist_train = window_attention_dist(reduced_attention_train, bias_indices_train, window_size=7, num_concat=len(params.intermediary_task["attention"]["layers"]))
attention_dist_train = windowed_dist_train

In [13]:
train_dataset.add_data(attention_dist_train, "attention_dist")
train_dataset.shuffle_data()
assert(attention_dist_train.shape[1] == params.final_task['input_dim'])

In [14]:
attention_scores_indexed_dev = return_idx_attention_dist(attention_scores_dev, bias_indices_dev)
reduced_attention_dev = reduce_attention_dist(attention_scores_indexed_dev, params.intermediary_task["attention"])
windowed_dist_dev = window_attention_dist(reduced_attention_dev, bias_indices_dev, window_size=7, num_concat=len(params.intermediary_task["attention"]["layers"]))
attention_dist_dev = windowed_dist_dev

In [15]:
dev_dataset.add_data(attention_dist_dev, "attention_dist")
dev_dataset.add_data(dev_dataset.get_val('bias_label'),'weak_bias_label')
dev_dataset.shuffle_data()
assert(attention_dist_dev.shape[1] == params.final_task['input_dim'])

### This is where the classification experiment starts

We create a classification experiment that contains useful methods for classifying bias based on the attention distributions. 

In [16]:
from sklearn.linear_model import LogisticRegression

In [18]:
train_dataset.get_val('attention_dist').shape

torch.Size([52275, 180])

In [19]:
X_train = train_dataset.get_val('attention_dist')
y_train = train_dataset.get_val('weak_bias_label')

In [20]:
X_dev = dev_dataset.get_val('attention_dist')
y_dev = dev_dataset.get_val('weak_bias_label')

In [21]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

/u/nlp/anaconda/main/anaconda3/envs/bias_classification/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [22]:
clf.score(X_dev, y_dev)

0.7037037037037037

In [25]:
cclf.coef_

(1, 180)